In [1]:
import pandas as pd

In [2]:
lang = 'eng'

In [3]:
path = "/pscratch/sd/c/chubakov/code/morphology/gpu/neural-transducer/checkpoints/transformer/tagtransformer/sigmorphon17-task1-dropout0.3/{lang}-high-.decode.dev.tsv".format(lang=lang)
path

'/pscratch/sd/c/chubakov/code/morphology/gpu/neural-transducer/checkpoints/transformer/tagtransformer/sigmorphon17-task1-dropout0.3/eng-high-.decode.dev.tsv'

In [4]:
df = pd.read_csv(path, sep='\t')

In [5]:
df

,prediction,target,loss,dist
0,w o n ' t y o u h a v e b e e n a n s ...,w o n ' t y o u h a v e b e e n a n s ...,0.050913,0
1,a r e w e a n s w e r i n g ?,a r e w e a n s w e r i n g ?,0.050941,0
2,h a s n ' t h e a n s w e r e d i t ?,h a s n ' t h e a n s w e r e d i t ?,0.056438,0
3,w o u l d n ' t i t a n s w e r y o u ?,w o u l d n ' t i t a n s w e r y o u ?,0.055186,0
4,I a n s w e r e d .,I a n s w e r e d .,0.048080,0
...,...,...,...,...
995,w i l l w e b e w a n t i n g h i m ...,w i l l w e b e w a n t i n g h i m ...,0.048174,0
996,w i l l i t b e w a n t i n g h i m ...,w i l l i t b e w a n t i n g h i m ...,0.048106,0
997,w o u l d i t w a n t y o u f r o m ...,w o u l d i t w a n t y o u f r o m ...,0.046933,0
998,h a s s h e b e e n w a n t i n g h i ...,h a s s h e b e e n w a n t i n g h i ...,0.047955,0


In [6]:
def fix_spacing(sentence):
    
    sentence = sentence.replace("   ", "_")
    sentence = sentence.replace(" ", "")
    sentence = sentence.replace("_", " ")
    
    return sentence
    
df['prediction'] = df.apply(lambda x: fix_spacing(x.prediction), axis=1)
df['target'] = df.apply(lambda x: fix_spacing(x.target), axis=1)

In [7]:
df

,prediction,target,loss,dist
0,won't you have been answering yourself?,won't you have been answering yourself?,0.050913,0
1,are we answering?,are we answering?,0.050941,0
2,hasn't he answered it?,hasn't he answered it?,0.056438,0
3,wouldn't it answer you?,wouldn't it answer you?,0.055186,0
4,I answered.,I answered.,0.048080,0
...,...,...,...,...
995,will we be wanting him from you?,will we be wanting him from you?,0.048174,0
996,will it be wanting him from us?,will it be wanting him from us?,0.048106,0
997,would it want you from yourself?,would it want you from yourself?,0.046933,0
998,has she been wanting him from me?,has she been wanting him from me?,0.047955,0


In [8]:
df['clause_match'] = df['prediction'] == df['target']
df["clause_match"] = df["clause_match"].astype(int)

In [9]:
def word_level_cmp(prediction, target):    
    
    words1 = prediction.split(' ')
    words2 = target.split(' ')
    
    # print(len(words1), len(words2))
    
    num_correct = 0
    
    min_l = min(len(words1), len(words2))
    
    for i in range(min_l):
        if words1[i] == words2[i]:
            num_correct += 2
            
    return num_correct / (len(words1) + len(words2))    

In [10]:
df['word_match'] = df.apply(lambda x: word_level_cmp(x.prediction, x.target), axis=1)

In [11]:
df

,prediction,target,loss,dist,clause_match,word_match
0,won't you have been answering yourself?,won't you have been answering yourself?,0.050913,0,1,1.0
1,are we answering?,are we answering?,0.050941,0,1,1.0
2,hasn't he answered it?,hasn't he answered it?,0.056438,0,1,1.0
3,wouldn't it answer you?,wouldn't it answer you?,0.055186,0,1,1.0
4,I answered.,I answered.,0.048080,0,1,1.0
...,...,...,...,...,...,...
995,will we be wanting him from you?,will we be wanting him from you?,0.048174,0,1,1.0
996,will it be wanting him from us?,will it be wanting him from us?,0.048106,0,1,1.0
997,would it want you from yourself?,would it want you from yourself?,0.046933,0,1,1.0
998,has she been wanting him from me?,has she been wanting him from me?,0.047955,0,1,1.0


In [12]:
print('current language: ', lang)

current language:  eng


In [13]:
df.describe()

,loss,dist,clause_match,word_match
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.068838,0.158000,0.946000,0.985155
std,0.113476,0.975699,0.226131,0.067076
min,0.045165,0.000000,0.000000,0.400000
25%,0.048281,0.000000,1.000000,1.000000
50%,0.049358,0.000000,1.000000,1.000000
75%,0.050316,0.000000,1.000000,1.000000
max,1.433519,14.000000,1.000000,1.000000


In [14]:
out_path = "/pscratch/sd/c/chubakov/code/morphology/gpu/neural-transducer/output/{lang}-output.dev.tsv".format(lang=lang)
out_path

df.to_csv(out_path, sep='\t')

In [15]:
eval_out_path = "/pscratch/sd/c/chubakov/code/morphology/evaluation/res/{lang}.dev".format(lang=lang)
eval_out_path

eval_df = df[['prediction']]

eval_df.to_csv(eval_out_path, index=False, header=False, sep='\t')